In [5]:
import torch
import torch.nn as nn
from torchvision import models
import sys
sys.path.append('../')
from pyqcr.torchgraph.tracer import TorchTracer
from pyqcr.transformation.module_fusion import Fuser
from pyqcr.quantization import compatability as comp

In [2]:
model = models.resnet18()
inp = torch.rand((1,3,224,224))
with TorchTracer() as tt:
    tt.trace_model(model, inp)
    model.graph = tt.to_graph()

model.graph.to_namegraph()

{'Conv2d0': ['BatchNorm2d0'], 'Tensor0': ['Conv2d0'], 'BatchNorm2d0': ['ReLU0'], 'ReLU0': ['MaxPool2d0'], 'MaxPool2d0': ['Conv2d1', '__iadd__0'], 'Conv2d1': ['BatchNorm2d1'], '__iadd__0': ['ReLU2'], 'BatchNorm2d1': ['ReLU1'], 'ReLU1': ['Conv2d2'], 'Conv2d2': ['BatchNorm2d2'], 'BatchNorm2d2': ['__iadd__0'], 'ReLU2': ['Conv2d3', '__iadd__1'], 'Conv2d3': ['BatchNorm2d3'], '__iadd__1': ['ReLU4'], 'BatchNorm2d3': ['ReLU3'], 'ReLU3': ['Conv2d4'], 'Conv2d4': ['BatchNorm2d4'], 'BatchNorm2d4': ['__iadd__1'], 'ReLU4': ['Conv2d5', 'Conv2d7'], 'Conv2d5': ['BatchNorm2d5'], 'Conv2d7': ['BatchNorm2d7'], 'BatchNorm2d5': ['ReLU5'], 'ReLU5': ['Conv2d6'], 'Conv2d6': ['BatchNorm2d6'], 'BatchNorm2d6': ['__iadd__2'], '__iadd__2': ['ReLU6'], 'ReLU6': ['Conv2d8', '__iadd__3'], 'Conv2d8': ['BatchNorm2d8'], '__iadd__3': ['ReLU8'], 'BatchNorm2d7': ['__iadd__2'], 'BatchNorm2d8': ['ReLU7'], 'ReLU7': ['Conv2d9'], 'Conv2d9': ['BatchNorm2d9'], 'BatchNorm2d9': ['__iadd__3'], 'ReLU8': ['Conv2d10', 'Conv2d12'], 'Conv2d1

In [9]:
# Find all modules which used multiple times during forward. 
# If there is reused modules pytorch1.6 quantization won't work correctly.
comp.get_reused_modules(model)

{'graph_names': ['ReLU2',
  'ReLU4',
  'ReLU6',
  'ReLU8',
  'ReLU10',
  'ReLU12',
  'ReLU14',
  'ReLU16'],
 'model_names': ['layer1.0.relu',
  'layer1.1.relu',
  'layer2.0.relu',
  'layer2.1.relu',
  'layer3.0.relu',
  'layer3.1.relu',
  'layer4.0.relu',
  'layer4.1.relu']}

In [10]:
# In order to quantize pytorch model with 1.6 API we need to convert all non-module operations to modules.
comp.get_non_module_ops(model)

['__iadd__0',
 '__iadd__1',
 '__iadd__2',
 '__iadd__3',
 '__iadd__4',
 '__iadd__5',
 '__iadd__6',
 '__iadd__7',
 'flatten0']